In [1]:
#colab環境にgdriveをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
from keras.datasets import mnist
import cv2
(X_train, y_train), (X_test, y_test) = mnist.load_data()

#-1~+1に正規化
X_train = (X_train.astype(np.float32) - 127.5)/127.5


11493376/11490434 [==============================] - 0s 0us/step


In [ ]:

from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout,Reshape,Add
from keras.layers import BatchNormalization, Activation, ZeroPadding2D,MaxPooling2D,GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import cv2

import matplotlib.pyplot as plt
import sys
import numpy as np




class DCGAN():
    def __init__(self):
        # Input shape
        self.input_img_rows = 28
        self.input_img_cols = 28
        self.input_channels = 1
        self.input_img_shape = (self.input_img_rows, self.input_img_cols, self.input_channels,)

        self.latent_dim = 100

        optimizer = Adam(0.0001, 0.5)

        # 2値分類のための識別モデルを作成
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        
        self.generator = self.build_generator()
        z = Input(shape = self.latent_dim)
        img = self.generator(z)

        # discriminatorのパラメータ固定
        self.discriminator.trainable = False
        valid = self.discriminator(img)

        self.combined = Model(z, valid)#並列結合
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):
        k=2 #成長率(笑)
        filter=64

        #imgの形に合わせて作られる
        input = Input(shape=self.latent_dim)
        x = Dense(filter*7*7, input_shape=[self.latent_dim])(input)
        x = Reshape((7,7,filter))(x)
        
        x = Conv2D(filter,(3,3),padding="same")(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(alpha=0.2)(x)

        x2 = Conv2D(filter,(3,3),padding="same")(x)
        x2 = BatchNormalization()(x2)
        x2 = LeakyReLU(alpha=0.2)(x2)
        x2 = Conv2D(filter,(3,3),padding="same")(x2)
        x2 = BatchNormalization()(x2)
        x = Conv2D(filter,(3,3),padding="same")(x)
        x = Add()([x2,x])
        x = LeakyReLU(alpha=0.2)(x)
        x = UpSampling2D((2,2))(x)

        filter/=k

        x2 = Conv2D(filter,(3,3),padding="same")(x)
        x2 = BatchNormalization()(x2)
        x2 = LeakyReLU(alpha=0.2)(x2)
        x2 = Conv2D(filter,(3,3),padding="same")(x2)
        x2 = BatchNormalization()(x2)
        x = Conv2D(filter,(3,3),padding="same")(x)
        x = Add()([x2,x])
        x = LeakyReLU(alpha=0.2)(x)
        x = UpSampling2D((2,2))(x)

        filter/=k

        x2 = Conv2D(filter,(3,3),padding="same")(x)
        x2 = BatchNormalization()(x2)
        x2 = LeakyReLU(alpha=0.2)(x2)
        x2 = Conv2D(filter,(3,3),padding="same")(x2)
        x2 = BatchNormalization()(x2)
        x = Conv2D(filter,(3,3),padding="same")(x)
        x = Add()([x2,x])
        x = LeakyReLU(alpha=0.2)(x)

        x = Conv2D(filter,(3,3),padding="same")(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(alpha=0.2)(x)

        x = Dense(self.input_channels)(x)

        output = Activation('tanh')(x)
        model = Model(input,output)

        model.summary()
        return model

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(256, kernel_size=3, strides=1, input_shape=self.input_img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Conv2D(128, kernel_size=3, strides=1, input_shape=self.input_img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Conv2D(64, kernel_size=3, strides=1, input_shape=self.input_img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Conv2D(32, kernel_size=3, strides=1, input_shape=self.input_img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))



        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.input_img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=500):
        print("実行1")
        
        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        #コンスタントなノイズ
        imagegennoise = np.random.uniform(-1, 1, (32,self.latent_dim))

        for epoch in range(epochs):

            # 本物の画像をbatchの数だけランダムで持ってきます
            idx = np.random.randint(0, len(X_train), batch_size)
            true_imgs = X_train[idx]

            # batchの数だけノイズを生成し、generatorに画像を生成させる
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # 本物の画像とフェイク画像を識別機に学習させます。
            d_loss_real = self.discriminator.train_on_batch(true_imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # 誤差を伝搬させて、generatorを学習させます
            g_loss = self.combined.train_on_batch(noise, valid)

            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            if epoch % 50 == 0:
                testimg = self.generator.predict(imagegennoise)
                dst = np.array((testimg * 0.5 + 0.5) * 255, np.int32) #0~255(人間に見やすく)
                #32,28,28,1
                dst = dst.reshape(4,8,1,28,28)
                dst = dst.transpose(0,3,1,4,2)
                dst = dst.reshape(4*28,8*28,1)
                cv2.imwrite(f'/content/drive/MyDrive/GANstudy/2.12/createimages/restest-{str(epoch)}.jpg', dst)


if __name__ == '__main__':
    dcgan = DCGAN()
    dcgan.train(epochs=100000, batch_size=128, save_interval=500)

ストリーミング出力は最後の 5000 行に切り捨てられました。
29745 [D loss: 0.627990, acc.: 67.58%] [G loss: 0.915899]
29746 [D loss: 0.658157, acc.: 66.80%] [G loss: 0.869415]
29747 [D loss: 0.630875, acc.: 64.45%] [G loss: 0.906927]
29748 [D loss: 0.625052, acc.: 63.28%] [G loss: 0.900543]
29749 [D loss: 0.657796, acc.: 62.89%] [G loss: 0.873877]
29750 [D loss: 0.615865, acc.: 64.84%] [G loss: 0.940317]
29751 [D loss: 0.618764, acc.: 67.97%] [G loss: 0.927210]
29752 [D loss: 0.612292, acc.: 62.11%] [G loss: 0.951121]
29753 [D loss: 0.647295, acc.: 60.16%] [G loss: 0.920255]
29754 [D loss: 0.648082, acc.: 61.33%] [G loss: 0.877750]
29755 [D loss: 0.641099, acc.: 60.94%] [G loss: 0.939414]
29756 [D loss: 0.645812, acc.: 61.72%] [G loss: 0.917530]
29757 [D loss: 0.615373, acc.: 65.62%] [G loss: 0.948120]
29758 [D loss: 0.653952, acc.: 58.98%] [G loss: 0.912829]
29759 [D loss: 0.643555, acc.: 62.89%] [G loss: 0.912549]
29760 [D loss: 0.661627, acc.: 62.50%] [G loss: 0.932870]
29761 [D loss: 0.624671, acc.: 64.06%] [